In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['content_last'] = mood.groupby('user_id')['content'].shift()
mood.loc[(pd.isnull(mood.content_last)), 'content_last'] = mood['content']
mood_content = mood.drop(["anxious", "bored", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
mood_content[:5]

,content,day_time_window,average_TimeUse,bulk,messaging,socialnetworking,otherapp,content_last
4825,4.0,3.0,0.673000,0,0,0,1,4.0
4826,4.0,4.0,64.314667,0,1,0,2,4.0
245,4.0,1.0,20.590500,0,0,0,2,4.0
4827,4.0,2.0,0.000000,0,0,0,0,4.0
4828,4.0,3.0,53.957200,0,3,0,2,4.0


In [4]:
# Normalize predictors
mood_normz = mood_content.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [5]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

#score = make_scorer(macroaverage_mae, greater_is_better=False)

In [6]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:-1]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [7]:
X_train, y_train = shuffle(X, y)

In [8]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [10]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['content'].ravel()) 
    X_use, y_use = shuffle(X_train_oversampl, y_train_oversampl)
    
    model = Sequential()
    model.add(Dense(24, input_dim=6, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_use, y_use, epochs=100, batch_size=512, verbose=0)
    y_pred = model.predict(X_test)
    
    pred = []
    for a in y_pred:
        for b in a:
            pred.append(round(b))
            
    print("Predicted values: ", Counter(pred))
    f1_scores.append(round(f1_score(y_test, pred, average='weighted'), 4))
    f1_scores_classes.append(f1_score(y_test, pred, average=None))
    
    test = y_test.values
    mmae_scores.append(macroaverage_mae(test, pred))

Predicted values:  Counter({2.0: 1012, 3.0: 295, 4.0: 12, 1.0: 4})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2.0: 1028, 3.0: 285, 1.0: 5, 4.0: 4})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2.0: 864, 3.0: 445, 4.0: 13})


...

/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2.0: 1004, 3.0: 306, 1.0: 6, 4.0: 6})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted values:  Counter({2.0: 802, 3.0: 515, 1.0: 3, 4.0: 2})


/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.0, 0.0061, 0.3067, 0.3409, 0.0139, 0.0]
STD F1-score per class:  [0.0, 0.0097, 0.038, 0.062, 0.0128, 0.0]


In [12]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.1857, 0.1517, 0.1833, 0.1502, 0.1842, 0.1652, 0.1948, 0.1781, 0.1517, 0.1662]
Mean MA F1-scores:  0.1711
Std MA F1-scores:  0.0155


In [13]:
print("Macro-averaged MAE: ", mmae_scores)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [array([1.43084875]), array([1.45260288]), array([1.4596063]), array([1.46407026]), array([1.49575932]), array([1.47903425]), array([1.47080781]), array([1.48567199]), array([1.47631226]), array([1.46470143])]
Mean MMAE-scores:  1.4679
Std MMAE-scores:  0.0173
